In [1]:
import asyncio
from hfc.fabric import Client
import nest_asyncio
nest_asyncio.apply()

In [2]:
loop = asyncio.get_event_loop()

cli = Client(net_profile="test/fixtures/network.json")
org1_admin = cli.get_user(org_name='org1.example.com', name='Admin')

Init client with profile=test/fixtures/network.json
create org with name=orderer.example.com
create org with name=org1.example.com
create org with name=org2.example.com
create ca with name=ca-org1
create ca with name=ca-org2
Import orderers = dict_keys(['orderer.example.com'])
Import peers = dict_keys(['peer0.org1.example.com', 'peer1.org1.example.com', 'peer0.org2.example.com', 'peer1.org2.example.com'])


In [3]:
# Create a New Channel, the response should be true if succeed
response = loop.run_until_complete(cli.channel_create(
            orderer='orderer.example.com',
            channel_name='testchannel',
            requestor=org1_admin,
            config_yaml='test/fixtures/e2e_cli/',
            channel_profile='TwoOrgsChannel'
            ))
print(response)
print(response == True)

FABRIC_CFG_PATH set to /home/prixa-ml/PycharmProjects/goProject/fabric-sdk-py/test/fixtures/e2e_cli/
Configtx file successfully created in current             directory
{'tx_id': 'aeb1525b6c6dede3b0b85f4beaf7b90a2c45395e8019a66a451050a52e506a3e', 'nonce': b'\xec\xaa-h\xd5J\xa5\x87@\t\xe4\xcdU0v/3bP\xf7C\xe1Ei', 'signatures': [b'\n\xd3\x06\n\xb6\x06\n\x07Org1MSP\x12\xaa\x06-----BEGIN CERTIFICATE-----\nMIICKjCCAdCgAwIBAgIQEn3uLYlL4sXXQBS1/k8u7zAKBggqhkjOPQQDAjBzMQsw\nCQYDVQQGEwJVUzETMBEGA1UECBMKQ2FsaWZvcm5pYTEWMBQGA1UEBxMNU2FuIEZy\nYW5jaXNjbzEZMBcGA1UEChMQb3JnMS5leGFtcGxlLmNvbTEcMBoGA1UEAxMTY2Eu\nb3JnMS5leGFtcGxlLmNvbTAeFw0xODEwMTkwMzQ4MDBaFw0yODEwMTYwMzQ4MDBa\nMGwxCzAJBgNVBAYTAlVTMRMwEQYDVQQIEwpDYWxpZm9ybmlhMRYwFAYDVQQHEw1T\nYW4gRnJhbmNpc2NvMQ8wDQYDVQQLEwZjbGllbnQxHzAdBgNVBAMMFkFkbWluQG9y\nZzEuZXhhbXBsZS5jb20wWTATBgcqhkjOPQIBBggqhkjOPQMBBwNCAAR2zFPDBYIy\njZSXaB15ILW9MRUYkSiksD6Io+VBbsJex2S7Do7lJhYfpyg5Z1LEHLCQgHH/VJ1F\nMZmNHgIIHqWPo00wSzAOBgNVHQ8BAf8EBAMCB4AwDAYDVR0TAQH/BAIwADArBgNV\nHS

True
True


In [4]:
orderer_admin = cli.get_user(org_name='orderer.example.com', name='Admin')
responses = loop.run_until_complete(cli.channel_join(
               requestor=org1_admin,
               channel_name='testchannel',
               peers=['peer0.org1.example.com',
                      'peer1.org1.example.com'],
               orderer='orderer.example.com'
               ))

print(responses)
print(len(responses) == 2)

get genesis block successfully, block=data_hash: "\335\002\250\2230\217\302\231A\tVG\336&\243+\312\221\214\031\001$\024\213\321e5\217C\007:\320"



[response {
  status: 200
}
, response {
  status: 200
}
]
True


In [5]:
cli.new_channel('testchannel')

New channel with name = testchannel


In [6]:
# Install Example Chaincode to Peers
# GOPATH setting is only needed to use the example chaincode inside sdk
import os
gopath_bak = os.environ.get('GOPATH', '')
gopath = os.path.normpath(os.path.join(
                      os.path.dirname(os.path.realpath('__file__')),
                      'test/fixtures/chaincode'
                     ))
os.environ['GOPATH'] = os.path.abspath(gopath)
# os.environ['GOPATH'] = os.path.abspath('/home/prixa-ml/go')

In [7]:
# The response should be true if succeed
responses = loop.run_until_complete(cli.chaincode_install(
               requestor=org1_admin,
               peers=['peer0.org1.example.com',
                      'peer1.org1.example.com'],
               cc_path='github.com/example_cc',
               cc_name='example_cc',
               cc_version='v1.0'
               ))

responses

[response {
   status: 200
   payload: "OK"
 },
 response {
   status: 200
   payload: "OK"
 }]

In [8]:
# Instantiate Chaincode in Channel, the response should be true if succeed
args = ['ram', '10000', 'not-ram', '30000']


policy = {
    'identities': [
        {'role': {'name': 'member', 'mspId': 'Org1MSP'}},
    ],
    'policy': {
        '1-of': [
            {'signed-by': 0},
        ]
    }
}

In [10]:
response = loop.run_until_complete(cli.chaincode_instantiate(
               requestor=org1_admin,
               channel_name='testchannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='example_cc',
               cc_version='v1.0',
               cc_endorsement_policy=policy, # optional, but recommended
               collections_config=None, # optional, for private data policy
               transient_map=None, # optional, for private data
               wait_for_event=True # optional, for being sure chaincode is instantiated
               ))
response

x is ------------>>> version: 1
response {
  status: 200
  payload: "\n\nexample_cc\022\004v1.0\032\004escc\"\004vscc*\027\022\010\022\006\010\001\022\002\010\000\032\013\022\t\n\007Org1MSP2D\n \261\374\334Tl8B\013[A\001\3334\342\335\255^\256\352>\301dU\3741\256\264\257~\255\231\341\022 zu\262KC;A\273\263S\3363b\364/\001L\372\346\260\324\253\341\001\025\036\257\036mg\272\215: \307\300\343\333VUH\035O\201 \356\360\347\342\352\025LZ\306\205\375\320t\006\007Q\235\203\006*\352B,\022\014\022\n\010\001\022\002\010\000\022\002\010\001\032\r\022\013\n\007Org1MSP\020\001\032\r\022\013\n\007Org2MSP\020\001"
}
payload: "\n \314H\376\232pE_\305\013\364i\2043\023\\\005d\271\303\203(\307\220\346\2634\037-\345Y\317\000\022\221\004\n\251\002\022.\n\nexample_cc\022 \032\020\n\007not-ram\032\00530000\032\014\n\003ram\032\00510000\022\366\001\n\004lscc\022\355\001\n\014\n\nexample_cc\032\334\001\n\nexample_cc\032\315\001\n\nexample_cc\022\004v1.0\032\004escc\"\004vscc*\027\022\010\022\006\010\001\022\002

{'name': 'example_cc',
 'version': 'v1.0',
 'escc': 'escc',
 'vscc': 'vscc',
 'policy': {'version': 0,
  'rule': {'n_out_of': {'n': 1, 'rules': [{'signed_by': 0}]}},
  'identities': [{'principal_classification': 'ROLE',
    'principal': {'msp_identifier': 'Org1MSP', 'role': 'MEMBER'}}]},
 'data': {'hash': b'\xb1\xfc\xdcTl8B\x0b[A\x01\xdb4\xe2\xdd\xad^\xae\xea>\xc1dU\xfc1\xae\xb4\xaf~\xad\x99\xe1',
  'metadatahash': b'zu\xb2KC;A\xbb\xb3S\xde3b\xf4/\x01L\xfa\xe6\xb0\xd4\xab\xe1\x01\x15\x1e\xaf\x1emg\xba\x8d'},
 'id': b'\xc7\xc0\xe3\xdbVUH\x1dO\x81 \xee\xf0\xe7\xe2\xea\x15LZ\xc6\x85\xfd\xd0t\x06\x07Q\x9d\x83\x06*\xea',
 'instantiation_policy': {'version': 0,
  'rule': {'n_out_of': {'n': 1,
    'rules': [{'signed_by': 0}, {'signed_by': 1}]}},
  'identities': [{'principal_classification': 'ROLE',
    'principal': {'msp_identifier': 'Org1MSP', 'role': 'ADMIN'}},
   {'principal_classification': 'ROLE',
    'principal': {'msp_identifier': 'Org2MSP', 'role': 'ADMIN'}}]}}

In [13]:
# Invoke a chaincode
args = ['ram', 'not-ram', '100']
# The response should be true if succeed
response = loop.run_until_complete(cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='testchannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='example_cc',
               transient_map=None, # optional, for private data
               wait_for_event=True, # for being sure chaincode invocation has been commited in the ledger, default is on tx event
               #cc_pattern='^invoked*' # if you want to wait for chaincode event and you have a `stub.SetEvent("invoked", value)` in your chaincode
               ))

response

'30100'

In [14]:
from hfc.fabric.channel.channel import Channel

In [24]:
Channel('testchannel',cli)._client

In [35]:
from hfc.fabric.transaction.tx_proposal_request import create_tx_prop_req,TXProposalRequest
from hfc.util.consts import SYSTEM_CHANNEL_NAME, CC_INSTANTIATE, CC_UPGRADE, CC_INVOKE, CC_QUERY, CC_TYPE_GOLANG

In [ ]:
from hfc.fabric.transaction import tx_proposal_request

In [36]:
def create_tx_prop_req(prop_type=None, cc_path=None, cc_type=CC_TYPE_GOLANG,
                       cc_name=None, cc_version=None, fcn=None, args=None,
                       cc_endorsement_policy=None,
                       transient_map=None, packaged_cc=None,
                       collections_config=None, package_id=None, is_init=False):
    """Create a transaction proposal request

    :param prop_type: proposal request type (Default value = None)
    :param cc_path: chaincode path (Default value = None)
    :param cc_name: chaincode name (Default value = None)
    :param cc_version: chaincode version (Default value = None)
    :param fcn: function name (Default value = None)
    :param args: function arguments (Default value = None)
    :param cc_endorsement_policy: chaincode endorsement policy (Default value = None)
    :param transient_map: transient data map (Default value = None)
    :param packaged_cc: packaged chaincode source
    :param cc_type:  (Default value = CC_TYPE_GOLANG)
    :param collections_config:  (Default value = None)
    :return: a transaction proposal request (Default value = None)

    """
    tx_prop_req = TXProposalRequest(
        prop_type, cc_path, cc_type, cc_name, cc_version, fcn,
        args, cc_endorsement_policy, transient_map,
        packaged_cc, collections_config, package_id, is_init)
    return tx_prop_req

In [37]:
tx_prop_req = create_tx_prop_req(
            prop_type=CC_QUERY,
            fcn='GetChainInfo',
            cc_name='example_cc',
            args=['testchannel'],
            cc_type=CC_TYPE_GOLANG,
            transient_map=None)

In [39]:
tx_prop_req

In [44]:
tx_prop_req._fcn

'GetChainInfo'

In [48]:
from hfc.fabric.transaction.tx_context import TXContext, create_tx_context


In [49]:
requestor = cli.get_user('org1.example.com', 'Admin')

In [50]:
tx_context = create_tx_context(requestor, requestor.cryptoSuite,tx_prop_req)

In [54]:
tx_context._user 

In [55]:
peers=['peer0.org1.example.com']

In [57]:
target_peers = cli.get_target_peers(peers)

In [59]:
channel = cli.get_channel('testchannel')

In [65]:
responses, proposal, header = channel.query_info(tx_context,target_peers)

In [66]:
responses

[<coroutine object _UnaryUnaryMultiCallable.__call__ at 0x7f5480667dc0>]

In [67]:
res = await asyncio.gather(*responses)
res

[version: 1
 response {
   status: 200
   payload: "\010\003\022 \324\264:\212=\2632\214}\022\354+\037\357jHp\357\"z\214g\363\201y\346\355^KG\253i\032 \036\246\021\337\331N0L\036\275\324gW\271\374Lzd\037\372\311\037g\223\3742\312L&us\t"
 }
 payload: "\n 0.-O4\001_9\267U\031q\364\232\004\210J\311\276\224B\275\315\325\256\270\031\016s\300\204\362\022\\\032K\010\310\001\032F\010\003\022 \324\264:\212=\2632\214}\022\354+\037\357jHp\357\"z\214g\363\201y\346\355^KG\253i\032 \036\246\021\337\331N0L\036\275\324gW\271\374Lzd\037\372\311\037g\223\3742\312L&us\t\"\r\022\004qscc\032\0051.4.6"
 endorsement {
   endorser: "\n\007Org1MSP\022\252\006-----BEGIN CERTIFICATE-----\nMIICKDCCAc+gAwIBAgIRAK5JdC2UorbI3GYUD7Z+I08wCgYIKoZIzj0EAwIwczEL\nMAkGA1UEBhMCVVMxEzARBgNVBAgTCkNhbGlmb3JuaWExFjAUBgNVBAcTDVNhbiBG\ncmFuY2lzY28xGTAXBgNVBAoTEG9yZzEuZXhhbXBsZS5jb20xHDAaBgNVBAMTE2Nh\nLm9yZzEuZXhhbXBsZS5jb20wHhcNMTgxMDE5MDM0ODAwWhcNMjgxMDE2MDM0ODAw\nWjBqMQswCQYDVQQGEwJVUzETMBEGA1UECBMKQ2FsaWZvcm5pYTEWMBQGA1UEBxMN\